In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("KafkaToConsole") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/18 12:51:25 WARN Utils: Your hostname, Izabellas-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.166.165 instead (on interface en0)
26/01/18 12:51:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/izabellasvavolya/.ivy2.5.2/cache
The jars for the packages stored in: /Users/izabellasvavolya/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
org.apache.spark#spark-token-provider-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e45c02b4-e9e8-4f47-9b10-6160b3aa7e4c;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.1.1 in central
	found org.apache.spark#spark-token

In [6]:
from pyspark.sql.functions import col

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "electric_cars") \
    .option("startingOffsets", "earliest") \
    .load()

out = df.select(col("value").cast("string").alias("value"))

q = out.writeStream \
    .format("console") \
    .outputMode("append") \
    .option("truncate", "false") \
    .start()


-------------------------------------------
Batch: 0
-------------------------------------------


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                        

-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                       

In [9]:
from pyspark.sql.functions import get_json_object, count, desc

parsed = out.select(
    get_json_object(col("value"), "$.City").alias("City"),
    get_json_object(col("value"), "$.Model Year").cast("int").alias("ModelYear")
)

top3 = parsed.filter(col("ModelYear") == 2023) \
    .groupBy("City") \
    .agg(count("*").alias("num_cars")) \
    .orderBy(desc("num_cars")) \
    .limit(3)


In [10]:
from pyspark.sql.functions import col, get_json_object, count, desc

parsed = out.select(
    get_json_object(col("value"), "$.City").alias("City"),
    get_json_object(col("value"), "$.Model Year").cast("int").alias("ModelYear")
)

top3 = parsed.filter(col("ModelYear") == 2023) \
    .groupBy("City") \
    .agg(count("*").alias("num_cars")) \
    .orderBy(desc("num_cars")) \
    .limit(3)


In [12]:
q2 = top3.writeStream \
    .format("console") \
    .outputMode("complete") \
    .option("truncate", "false") \
    .start()


-------------------------------------------
Batch: 0
-------------------------------------------
+--------+--------+
|City    |num_cars|
+--------+--------+
|Seattle |6350    |
|Tukwila |5477    |
|Bellevue|3889    |
+--------+--------+



-------------------------------------------
Batch: 0
-------------------------------------------
+--------+--------+
|City    |num_cars|
+--------+--------+
|Seattle |6350    |
|Tukwila |5477    |
|Bellevue|3889    |
+--------+--------+

